In [2]:
import glob
import matplotlib.pyplot as plt
import mne
import numpy as np
import pyxdf

In [3]:
# load baseline file names
baseline_list = []
path = 'data/**/*S001*.xdf'
for file in glob.glob(path, recursive=True):
    baseline_list.append(file)
print(baseline_list[0])

data/gme2022/sub-Tobias/ses-S001/eeg/sub-Tobias_ses-S001_task-Default_run-001_eeg.xdf


In [4]:
# load experiment file names
experiment_list = []
path = 'data/**/*S002*.xdf'
for file in glob.glob(path, recursive=True):
    experiment_list.append(file)
print(experiment_list[0])


data/gme2022/sub-Tobias/ses-S002/eeg/sub-Tobias_ses-S002_task-Default_run-001_eeg.xdf


In [5]:
# showcase data content (freq, etc.)
streams, _ = pyxdf.load_xdf(baseline_list[0])
data = streams[1]["time_series"].T[:8]
stamps = streams[1]["time_stamps"][:8]
sfreq = float(streams[1]["info"]["nominal_srate"][0])
info = mne.create_info(8, sfreq, ["eeg"] * 8)
raw = mne.io.RawArray(data, info)
n_time_samps = raw.n_times
time_secs = raw.times
ch_names = raw.ch_names
n_chan = len(ch_names)  # note: there is no raw.n_channels attribute
print('the (cropped) sample data object has {} time samples and {} channels.'
    ''.format(n_time_samps, n_chan))
print('The last time sample is at {} seconds.'.format(time_secs[-1]))
print('The first few channel names are {}.'.format(', '.join(ch_names[:3])))
print()  # insert a blank line in the output

# some examples of raw.info:
print('bad channels:', raw.info['bads'])  # chs marked "bad" during acquisition
print(raw.info['sfreq'], 'Hz')            # sampling frequency
print(raw.info['description'], '\n')      # miscellaneous acquisition info

print(raw.info)

Creating RawArray with float64 data, n_channels=8, n_times=17025
    Range : 0 ... 17024 =      0.000 ...    68.096 secs
Ready.
the (cropped) sample data object has 17025 time samples and 8 channels.
The last time sample is at 68.096 seconds.
The first few channel names are 0, 1, 2.

bad channels: []
250.0 Hz
None 

<Info | 7 non-empty values
 bads: []
 ch_names: 0, 1, 2, 3, 4, 5, 6, 7
 chs: 8 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 8
 projs: []
 sfreq: 250.0 Hz
>


In [6]:
# read baseline and cut it to 40 sec
sfreq = 250
info = mne.create_info(8, sfreq, ["eeg"] * 8)

baseline_mne_list = []
for baseline in baseline_list:
    streams, _ = pyxdf.load_xdf(baseline)
    try:
        data = streams[1]["time_series"].T[:8]
    except AttributeError:
        # some recordings miss the marker channel
        data = streams[0]["time_series"].T[:8]
    raw = mne.io.RawArray(data, info)
    raw = raw.crop(tmin=1, tmax=41)
    baseline_mne_list.append(raw)

print(baseline_mne_list[0])

Creating RawArray with float64 data, n_channels=8, n_times=17025
    Range : 0 ... 17024 =      0.000 ...    68.096 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=15271
    Range : 0 ... 15270 =      0.000 ...    61.080 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=10391
    Range : 0 ... 10390 =      0.000 ...    41.560 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=10893
    Range : 0 ... 10892 =      0.000 ...    43.568 secs
Ready.


Stream 3: Calculated effective sampling rate 0.0000 Hz is different from specified rate 250.0000 Hz.


Creating RawArray with float64 data, n_channels=8, n_times=16392
    Range : 0 ... 16391 =      0.000 ...    65.564 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=13529
    Range : 0 ... 13528 =      0.000 ...    54.112 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=15273
    Range : 0 ... 15272 =      0.000 ...    61.088 secs
Ready.


Stream 1: Calculated effective sampling rate 0.0000 Hz is different from specified rate 250.0000 Hz.
Stream 4: Calculated effective sampling rate 0.0000 Hz is different from specified rate 250.0000 Hz.


Creating RawArray with float64 data, n_channels=8, n_times=12646
    Range : 0 ... 12645 =      0.000 ...    50.580 secs
Ready.


Stream 3: Calculated effective sampling rate 0.0000 Hz is different from specified rate 250.0000 Hz.


Creating RawArray with float64 data, n_channels=8, n_times=15143
    Range : 0 ... 15142 =      0.000 ...    60.568 secs
Ready.
<RawArray | 8 x 10001 (40.0 s), ~638 kB, data loaded>


In [7]:
trails = { 
          'ball': list(),
          'ball+number': list(),
          'ball+number+wheel': list(),
         }

for experiment in experiment_list:
    print(experiment)
    streams, _ = pyxdf.load_xdf(experiment)
    marker = -1 
    eeg = -1
    for idx, stream in enumerate(streams):
        if streams[idx]['time_stamps'].shape[0] == 0:
            # ignore empty streams
            continue
        if stream["info"]["name"] == ["markers"]:
            marker = idx
        elif stream["info"]["name"] == ["eeg"]:
            eeg = idx
        else:
            raise Exception(f"stream info name unknown {stream['info']['name']}")

    if marker == -1 and eeg == -1:
        raise Exception('channels not found')    
    
    for i in range(0, streams[marker]["time_stamps"].shape[0], 2):
        event = streams[marker]["time_series"][i][0]
        if event in ['Starting Complex Eye Tracking Dashboard', 'Starting Simple Eye Tracking Dashboard', '']:
            print(f"Skip event {event}")
            continue
        start = streams[marker]["time_stamps"][i]
        stop = streams[marker]["time_stamps"][i+1]
        print(start, stop, event)

        trial = []
        for j, stamp in enumerate(streams[eeg]["time_stamps"]):
            if start <= stamp and stamp <= stop: 
                trial.append(streams[eeg]["time_series"][j])
        
        match event:
            case 'Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False':
                key = 'ball'
            case 'Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False':
                key = 'ball+number'
            case 'Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel True':
                key = 'ball+number+wheel'
            case _:
                print(f"key {event} not defined!")

        trial = np.array(trial)
        trails[key].append(trial)


data/gme2022/sub-Tobias/ses-S002/eeg/sub-Tobias_ses-S002_task-Default_run-001_eeg.xdf
101117.70974713018 101157.71034355162 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False
101211.09729437983 101251.10414130123 Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False
101297.27641387969 101337.28271760109 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel True
101400.86713255828 101440.87416547969 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False
101494.4609741065 101534.46757832791 Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False
101581.12291820301 101621.1297237244 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel True
101688.82983475293 101728.83650627433 Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False
101776.62455914155 101816.63162116293 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False
101875.08408965587 101915.09087207726 Starting Cognitive Lo

Stream 3: Calculated effective sampling rate 0.0000 Hz is different from specified rate 250.0000 Hz.


102572.11898197078 102612.11912452719 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False
102665.88939626861 102705.89619842502 Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False
102752.48466107425 102792.49142863067 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel True
102852.52675906527 102892.53369022167 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False
102944.4542208651 102984.4610386215 Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False
103031.49938467027 103071.50617492668 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel True
103133.42404775921 103173.43090001562 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False
103225.26804735915 103265.27515301555 Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False
103311.38044076532 103351.38718362173 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel True
Skip event Starting Simple Eye Tracking

Stream 1: Calculated effective sampling rate 0.0000 Hz is different from specified rate 250.0000 Hz.
Stream 4: Calculated effective sampling rate 0.0000 Hz is different from specified rate 250.0000 Hz.


104247.85696648824 104287.85686471527 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False
104338.57784554955 104378.58457677659 Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False
104426.12295680871 104466.12952163575 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel True
104528.24781037774 104568.25432980478 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False
104619.34177713931 104659.34861816635 Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False
104706.3206216981 104746.32724972513 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel True
104799.9807235614 104839.98770708844 Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False
104886.09290591959 104926.09965674665 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False
104975.1711951798 105015.17780100685 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel True
Skip event Starting Simple Eye Tracking 

Stream 3: Calculated effective sampling rate 0.0000 Hz is different from specified rate 250.0000 Hz.


96131.81327511564 96171.81979690412 Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False
96217.60865497649 96257.61551626495 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False
96306.75348622796 96346.76013941645 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel True
96432.32240197793 96472.3291769664 Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False
96518.51785203764 96558.5246451261 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False
96610.74516668054 96650.75211566902 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel True
96718.4522212803 96758.45884926878 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel False
96809.5297976264 96849.53626471487 Starting Cognitive Load Tasks: Balls True, Numbers False, Wheel False
96895.17520358764 96935.18193187611 Starting Cognitive Load Tasks: Balls True, Numbers True, Wheel True


In [8]:
print(len(trails["ball"]))
print(len(trails["ball+number"]))
print(len(trails["ball+number+wheel"]))

27
27
27


In [9]:
trails["ball"][0]

array([[2.1589434e+05, 1.9823003e+05, 3.0944741e+05, ..., 1.0000000e+02,
        6.6382000e+04, 1.0000000e+00],
       [2.1582362e+05, 1.9802189e+05, 3.0912653e+05, ..., 1.0000000e+02,
        6.6383000e+04, 1.0000000e+00],
       [2.1586002e+05, 1.9805309e+05, 3.0910319e+05, ..., 1.0000000e+02,
        6.6384000e+04, 1.0000000e+00],
       ...,
       [2.1585823e+05, 1.9805300e+05, 3.0392000e+05, ..., 1.0000000e+02,
        7.6392000e+04, 1.0000000e+00],
       [2.1588997e+05, 1.9820142e+05, 3.0417712e+05, ..., 1.0000000e+02,
        7.6393000e+04, 1.0000000e+00],
       [2.1584088e+05, 1.9810781e+05, 3.0410141e+05, ..., 1.0000000e+02,
        7.6394000e+04, 1.0000000e+00]], dtype=float32)